<a href="https://colab.research.google.com/github/sricinu/upgrad_rag/blob/main/upGrad_FINAL_Fine_Tuning_LoRA_Code%2B(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install --upgrade pip lamini --disable-pip-version-check torch torchdata datasets --quiet accelerate==0.20.3
! pip install accelerate huggingface_hub llama_index sentence_transformers logger
! pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 69.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━

In [ ]:
! pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.20.3
    Uninstalling accelerate-0.20.3:
      Successfully uninstalled accelerate-0.20.3


In [ ]:
! pip install transformers

In [ ]:
#!pip install -q peft==0.4.0 bitsandbytes==0.40.2 trl==0.4.7

In [ ]:
from llama import BasicModelRunner
from datasets import load_dataset
from transformers import GenerationConfig
from google.colab import userdata
from llama import LLMEngine
import os, torch, logging
from transformers import AutoModelForSeq2SeqLM, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline
from peft import LoraConfig, PeftModel
import pandas as pd
from transformers import Trainer
import datasets
from datasets import Dataset
import tempfile
import random
import os
import yaml
import time
#import transformers
import pandas as pd

import itertools
from pprint import pprint

# Customer Service dataset

In [ ]:
data_path = "https://raw.githubusercontent.com/bitext/customer-support-llm-chatbot-training-dataset/main/data/Bitext_Sample_Customer_Support_Training_Dataset_27K_responses-v11.csv"

In [ ]:
df_q_a = pd.read_csv(data_path)

In [ ]:
df_q_a

,flags,instruction,category,intent,response
0,B,question about cancelling order {{Order Number}},ORDER,cancel_order,I've understood you have a question regarding ...
1,BQZ,i have a question about cancelling oorder {{Or...,ORDER,cancel_order,I've been informed that you have a question ab...
2,BLQZ,i need help cancelling puchase {{Order Number}},ORDER,cancel_order,I can sense that you're seeking assistance wit...
3,BL,I need to cancel purchase {{Order Number}},ORDER,cancel_order,I understood that you need assistance with can...
4,BCELN,"I cannot afford this order, cancel purchase {{...",ORDER,cancel_order,I'm sensitive to the fact that you're facing f...
...,...,...,...,...,...
26867,BL,I am waiting for a rebate of {{Refund Amount}}...,REFUND,track_refund,Thank you for sharing your situation regarding...
26868,BIL,how to see if there is anything wrong with my ...,REFUND,track_refund,Ensuring the accuracy of your restitution is o...
26869,BLQZ,I'm waiting for a reimbjrsement of {{Currency ...,REFUND,track_refund,"Firstly, I genuinely understand the importance..."
26870,BL,I don't know what to do to see my reimbursemen...,REFUND,track_refund,I've understood you're unsure about how to che...


In [ ]:
print("question: " + df_q_a.iloc[0]["instruction"])
print("answer: " + df_q_a.iloc[0]["response"])

question: question about cancelling order {{Order Number}}
answer: I've understood you have a question regarding canceling order {{Order Number}}, and I'm here to provide you with the information you need. Please go ahead and ask your question, and I'll do my best to assist you.


In [ ]:
print("question: " + df_q_a.iloc[1]["instruction"])
print("answer: " + df_q_a.iloc[1]["response"])

question: i have a question about cancelling oorder {{Order Number}}
answer: I've been informed that you have a question about canceling order {{Order Number}}. I'm here to assist you! Please go ahead and let me know what specific question you have, and I'll provide you with all the information and guidance you need. Your satisfaction is my top priority.


## Prepare full dataset

In [ ]:
list_of_entry = []
for i in range(0, len(df_q_a)):
  entry = {
      'question': f"### Question:\n{df_q_a.iloc[i]['instruction']}\n\n### Answer:",
      'answer': df_q_a.iloc[i]['response']
  }
  list_of_entry.append(entry)

In [ ]:
list_of_entry[0]

{'question': '### Question:\nquestion about cancelling order {{Order Number}}\n\n### Answer:',
 'answer': "I've understood you have a question regarding canceling order {{Order Number}}, and I'm here to provide you with the information you need. Please go ahead and ask your question, and I'll do my best to assist you."}

In [ ]:
df_q_a = df_q_a.loc[df_q_a["intent"] == "cancel_order"]

In [ ]:
df_q_a

,flags,instruction,category,intent,response
0,B,question about cancelling order {{Order Number}},ORDER,cancel_order,I've understood you have a question regarding ...
1,BQZ,i have a question about cancelling oorder {{Or...,ORDER,cancel_order,I've been informed that you have a question ab...
2,BLQZ,i need help cancelling puchase {{Order Number}},ORDER,cancel_order,I can sense that you're seeking assistance wit...
3,BL,I need to cancel purchase {{Order Number}},ORDER,cancel_order,I understood that you need assistance with can...
4,BCELN,"I cannot afford this order, cancel purchase {{...",ORDER,cancel_order,I'm sensitive to the fact that you're facing f...
...,...,...,...,...,...
993,BL,I need help to cancel purchase {{Order Number}},ORDER,cancel_order,I'm fully aware that you're seeking assistance...
994,BL,I want help to cancel purchase {{Order Number}},ORDER,cancel_order,You've come to the right place! I understand t...
995,BIL,how to cancel purchase {{Order Number}}?,ORDER,cancel_order,I've decoded that you're looking for assistanc...
996,BLZ,I want help cancellkng purchase {{Order Number}},ORDER,cancel_order,I realized that you're seeking assistance with...


# LORA

In [ ]:
!pip install -q bitsandbytes datasets loralib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 25.4 MB/s eta 0:00:00


The environment variable CUDA_VISIBLE_DEVICES is used to manage the GPU device(s) that CUDA will see and can use when running a GPU-enabled application or library. This can be particularly useful in systems with multiple GPUs.

Purpose
The CUDA_VISIBLE_DEVICES environment variable is often set to configure the execution environment for GPU-accelerated applications to specify one or more GPUs that the CUDA runtime and the application should use.

The device_map parameter in the from_pretrained method of the AutoModelForCausalLM class is a feature that allows you to specify how the model's layers should be distributed across multiple devices, such as GPUs or TPUs.

When you're dealing with very large models like "bigscience/bloom-560m", the model might not fit into the memory of a single GPU, or you might want to utilize multiple GPUs for faster computation. The device_map parameter gives you the control to allocate the model's components to different devices as needed.

Here's a brief explanation of the device_map and how to use it with an example:

Overview
The device_map parameter is used to explicitly define the placement of the model's layers on different devices. This can help manage memory usage and computational load when working with large models or in a multi-GPU environment.

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "bigscience/bloom-560m",
    device_map="cuda:0",
)

tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-560m")

# Freeze original weights

In [ ]:
"""
Import the BloomForCausalLM class from the transformers library.
Functionality: The BloomForCausalLM class provides the implementation of
the BLOOM model specifically for causal language modeling tasks.
Usage: This is a prerequisite before you instantiate a model for any operations
like training, inference, or fine-tuning.
"""

from transformers import BloomForCausalLM

"""
Iterate over all the parameters in the model and freeze them.
Functionality: Setting requires_grad to False prevents the parameters from being
updated during training, effectively freezing the model.
Usage: This is commonly done when you want to fine-tune only certain parts of
the model (like task-specific adapters) and not the entire model.
"""

for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    # Within the loop, this conditional block casts parameters with one dimension to 32-bit floating point (FP32).
    # Casting parameters, often layer normalization parameters,
    # to FP32 can improve numerical stability during training.
    # Useful when the rest of your model uses lower-precision (like FP16)
    # for faster computation but requires certain parameters to be in FP32 to
    # prevent underflow or overflow.
    param.data = param.data.to(torch.float32)

#model.gradient_checkpointing_enable()  # reduce number of stored activations
"""
Enables gradients for model inputs.
This method is typically used when you need to compute gradients
with respect to inputs, for tasks such as adversarial training or visualization
techniques.
Call this method if you plan to calculate gradients on the inputs,
not typically required for standard training or inference.
"""
model.enable_input_require_grads()

"""
Defines a custom class CastOutputToFloat that casts the output of the model's
language model head to FP32.
Inherits from nn.Sequential for a sequence of operations, with an overridden
forward method that casts outputs to FP32.
Use this class to wrap any module whose output you want to ensure is in FP32,
particularly when your model operates in mixed precision mode.

"""

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)

"""
Replaces the original language model head of model with an instance of
CastOutputToFloat.
Ensures that the output of the language model head is in FP32,
even if the rest of the model uses mixed precision.
Assign this after instantiating the model, to ensure that outputs are
in the desired precision.
"""

model.lm_head = CastOutputToFloat(model.lm_head)

def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0 # just for initialization
    all_param = 0 # just for initialization
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()

    """
    Prints out the number of trainable parameters, the total
    number of parameters, and the percentage of parameters that are trainable.
    Usage: Call this function whenever you want to report on the model's
    parameter statistics, which can be critical for understanding the scale
    of your training task and for debugging purposes.
    """
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )


In [ ]:
from peft import LoraConfig, get_peft_model

"""
 Initialize an instance of LoraConfig, which is a configuration class for the
 LoRA (Low-Rank Adaptation) mechanism.

r: Specifies the rank of the low-rank matrices that will be used in
    the LoRA adaptation. This effectively controls the number of trainable
    parameters introduced by LoRA. A lower rank means fewer parameters and
    less capacity, whereas a higher rank introduces more capacity at the
    cost of more parameters.

lora_alpha: Defines the scaling factor (alpha) for the LoRA adaptation.
    This parameter is used to amplify the updates in the low-rank matrices and
    can have a significant impact on the effectiveness of the adaptation.

lora_dropout: Sets the dropout rate for the LoRA layers.
   Dropout is a regularization technique that involves randomly
   setting a fraction of input units to 0 at each update during training time,
   which helps to prevent overfitting.

bias: Determines how biases are handled in the LoRA adaptation. In this case,
   "none" indicates that no biases are being adapted or added in the LoRA modules.

task_type: Indicates the type of task the model is being configured for.
    "CAUSAL_LM" signifies that the model is set up for a causal language
    modeling task, meaning the model predicts the next token in
    a sequence while only having access to the previous tokens.

"""
config = LoraConfig(
    r=16, #attention heads
    lora_alpha=32, #alpha scaling
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM" # set this for CLM or Seq2Seq
)

"""
Applies the LoRA configuration to the provided model using the get_peft_model
function, which presumably adapts the model for parameter-efficient
fine-tuning according to the config settings.
"""
model = get_peft_model(model, config)

"""
This function iterates through all parameters in the model,
counts the total number of parameters, and the subset that is trainable,
then prints out these numbers along with the percentage of parameters that
are trainable.
Using this configuration and initialization process, you set up a model with
LoRA adaptations, which is then printed out to provide insight into
how many parameters are actually trainable after the adaptation.
This is a useful piece of information to evaluate the extent of the adaptation
and to ensure that the model configuration aligns with your computational
resources and training objectives.
"""
print_trainable_parameters(model)

trainable params: 1572864 || all params: 560787456 || trainable%: 0.2804741766549072


**The lora_alpha parameter** in the context of Low-Rank Adaptation (LoRA) is a scaling factor that is applied to the low-rank matrices which are inserted into the model as part of the adaptation process. This parameter is crucial for controlling the magnitude of changes introduced by the LoRA adaptation.

Here are more details about the role and impact of lora_alpha:

Purpose of lora_alpha
Scaling Updates: lora_alpha multiplies the updates that are applied to the weights of the model during the training process. If the updates computed by the optimization algorithm are small, scaling them up can help the model learn more quickly. Conversely, if the updates are too large, they can destabilize learning; hence a smaller lora_alpha might be preferable.

Controlling Learning Speed: By scaling the updates, lora_alpha indirectly controls the speed at which the model learns the adaptations. A higher lora_alpha accelerates the changes to the low-rank matrices, possibly leading to faster convergence, but also risking overshooting optimal solutions.

Balancing Adaptation and Stability: One of the challenges in adapting pre-trained models is to introduce enough flexibility to learn from new data while not deviating too much from what the model has already learned. lora_alpha helps balance this by controlling the extent to which LoRA alters the pre-existing weights.

Impact of lora_alpha
Effectiveness of LoRA: The right lora_alpha value can be crucial for the effectiveness of the LoRA adaptation. If it's too low, the model may not adapt sufficiently to new tasks. If it's too high, the model might overfit to the new task or forget previous knowledge (catastrophic forgetting).

Hyperparameter Tuning: lora_alpha is typically a hyperparameter that needs to be tuned based on the specific task and dataset. It may require experimentation to find the optimal value that balances training stability with the ability to adapt and improve on the new task.

Interaction with Other Hyperparameters: The effect of lora_alpha is not isolated; it interacts with other hyperparameters such as learning rate and r. When tuning lora_alpha, it's important to consider its relationship with these other parameters to ensure harmonious training dynamics.

In [ ]:
finetuning_dataset_loaded = Dataset.from_pandas(df_q_a[["instruction", "response"]])

def merge_function(entry):
    entry["full"] = entry["instruction"] + " " + entry["response"]

    return entry

finetuning_dataset_loaded = Dataset.from_pandas(df_q_a[["instruction", "response"]])
new = finetuning_dataset_loaded.map(
    merge_function
)

Map:   0%|          | 0/998 [00:00<?, ? examples/s]

In [ ]:
new

Dataset({
    features: ['instruction', 'response', '__index_level_0__', 'full'],
    num_rows: 998
})

In [ ]:
data = new.map(lambda samples: tokenizer(samples["full"]), batched=True)

Map:   0%|          | 0/998 [00:00<?, ? examples/s]

In [ ]:
data = data.add_column("labels", data["input_ids"])

In [ ]:
split_dataset = data.train_test_split(test_size=0.1, shuffle=True, seed=123)
print(split_dataset)

train_dataset = split_dataset["train"]
test_dataset = split_dataset["test"]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'response', '__index_level_0__', 'full', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 898
    })
    test: Dataset({
        features: ['instruction', 'response', '__index_level_0__', 'full', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
})


In [ ]:
split_dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'response', '__index_level_0__', 'full', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 898
    })
    test: Dataset({
        features: ['instruction', 'response', '__index_level_0__', 'full', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
})

In [ ]:
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA device name:", torch.cuda.get_device_name(0))


CUDA available: True
CUDA device name: Tesla T4


In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

In [ ]:
split_dataset["train"]

Dataset({
    features: ['instruction', 'response', '__index_level_0__', 'full', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 898
})

Importing the Transformers Library
Import the transformers package which provides classes for instantiating models, trainers, and handling datasets.

Creating a Trainer Instance
Instantiate a Trainer object from the transformers library, which will be used to train or fine-tune the model.

Setting the Model for Training
Assign the pre-initialized model that will be trained to the model argument of the Trainer.

Specifying the Training Dataset
Provide the training dataset part of the split_dataset to the train_dataset argument. The dataset should be split beforehand and contain the training data.

Configuring Training Arguments
Initialize TrainingArguments to set the hyperparameters for training the model.

Setting Batch Size Per Device
Define the batch size per device during training as 1. This dictates how many examples the model sees at once per device.

Configuring Gradient Accumulation
Set gradient_accumulation_steps to 1, indicating that the gradients are updated after each step.

Specifying Warmup Steps
Allocate 1 warmup step for learning rate scheduling, where the learning rate gradually increases to the specified maximum.

Limiting the Total Number of Training Steps
Restrict training to a total of 100 steps with max_steps.

Determining the Learning Rate
Set the initial learning rate for the optimizer as 2e-4.

Enabling Mixed Precision Training
Activate FP16 training mode for efficiency by setting fp16 to True. This uses less memory and can lead to faster training times on supported GPUs.

In [ ]:
import transformers
trainer = transformers.Trainer(
    model=model,
    train_dataset=split_dataset["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=1,
        warmup_steps=0,
        max_steps=200,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

You're using a BloomTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,2.479200
2,2.427500
3,2.852800
4,2.366500
5,2.253600
6,2.360300
7,2.212500
8,2.173800
9,2.144400
10,2.154300


TrainOutput(global_step=200, training_loss=1.1083051039278506, metrics={'train_runtime': 27.4276, 'train_samples_per_second': 7.292, 'train_steps_per_second': 7.292, 'total_flos': 87920394731520.0, 'train_loss': 1.1083051039278506, 'epoch': 0.22})

In [ ]:
def inference(text, model, tokenizer, max_input_tokens=1000, max_output_tokens=100):
  # Define the function 'inference' which takes a piece of text, a model, a tokenizer,
  # and optional arguments for the maximum number of input and output tokens.

  # Tokenize the input text.
  input_ids = tokenizer.encode(
          text,
          return_tensors="pt",  # Return the result as PyTorch tensors.
          truncation=True,  # Truncate the text to fit the max input length.
          max_length=max_input_tokens  # Set the maximum token length for the input text.
  )

  # Create an attention mask which is 1 for real tokens and 0 for padding tokens.
  attention_mask = (input_ids != tokenizer.pad_token_id).int()

  # Assign the model's device to a variable. This ensures that tensor operations
  # are performed on the same device where the model is loaded (CPU/GPU).
  device = model.device

  # Generate tokens using the model, passing in the processed inputs.
  generated_tokens_with_prompt = model.generate(
    input_ids=input_ids.to(device),  # Move input tokens to the same device as the model.
    max_length=max_output_tokens,  # Set the maximum token length for the output text.
    attention_mask=attention_mask,  # Provide the attention mask for the model to ignore padding.
    pad_token_id=tokenizer.eos_token_id  # Specify the end-of-sequence token ID for padding if needed.
  )

  # Decode the generated tokens back into text, ignoring special tokens like padding or end-of-sequence.
  generated_text_with_prompt = tokenizer.batch_decode(generated_tokens_with_prompt, skip_special_tokens=True)

  # Remove the initial prompt from the generated text to obtain only the model's output.
  generated_text_answer = generated_text_with_prompt[0][len(text):]

  # Return the final generated text, which is the answer from the model without the prompt.
  return generated_text_answer

In [ ]:
test_question = test_dataset[0]['instruction']
print("Question input (test):", test_question)

print("Finetuned slightly model's answer: ")
print(inference(test_question, model, tokenizer))

Question input (test): i need assistance with canceling purchase {{Order Number}}
Finetuned slightly model's answer: 


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


 I've been unable to locate the correct information regarding your purchase with the order number {{Order Number}}. I apologize for any inconvenience caused. Please contact our support team to resolve this issue. We are here to assist you and ensure your satisfaction. Please follow these steps:

1. Log in to your account: Access our platform by signing in to your {{Online Company Portal Info}}.
2. Navigate to your order history: Once logged in


# Save to Huggingface

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

# Push to Huggingface

In [ ]:
model.push_to_hub("Sapka/bloom-560m-customer",
                  use_auth_token=True,
                  commit_message="basic training",
                  private=True)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:853: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/6.30M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Sapka/bloom-560m-customer/commit/bd2e7c29d6ad35bee280bf74fcbf7f764832d849', commit_message='basic training', commit_description='', oid='bd2e7c29d6ad35bee280bf74fcbf7f764832d849', pr_url=None, pr_revision=None, pr_num=None)

# Import from Huggingface

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "Sapka/bloom-560m-customer"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)